# Krankenversicherung / Aufgabenserie vom 01.11.2024

## 1. Rückstellungsberechnung mit Chain-Ladder

Gegeben sei das folgende Abwicklungsdreieck mit den im Behandlungsmonat i angefallenen und im Abrechnugnsmonat k bezahlten Leistungen $S_{i,k}$:

|                   | Abrechnungs-monat k |        |        |        |
| ----------------- | ------------------- | ------ | ------ | ------ |
| Behandlungs-monat | 1                   | 2      | 3      | 4      |
| 1                 | 58000               | 276100 | 111700 | 20400  |
| 2                 |                     | 100900 | 480319 | 194319 |
| 3                 |                     |        | 120800 | 575050 |
| 4                 |                     |        |        | 130900 |

a) Berechnen Sie den Rückstellungsbedarf mit Hilfe des Chain-Ladder Verfahrens. 

In [1]:
library(tidyverse, quietly = TRUE, warn.conflicts = FALSE)

Warning message:
"Paket 'tidyverse' wurde unter R Version 4.2.3 erstellt"
Warning message:
"Paket 'ggplot2' wurde unter R Version 4.2.3 erstellt"
Warning message:
"Paket 'tibble' wurde unter R Version 4.2.3 erstellt"
Warning message:
"Paket 'tidyr' wurde unter R Version 4.2.3 erstellt"
Warning message:
"Paket 'readr' wurde unter R Version 4.2.3 erstellt"
Warning message:
"Paket 'purrr' wurde unter R Version 4.2.3 erstellt"
Warning message:
"Paket 'dplyr' wurde unter R Version 4.2.3 erstellt"
Warning message:
"Paket 'stringr' wurde unter R Version 4.2.3 erstellt"
Warning message:
"Paket 'forcats' wurde unter R Version 4.2.3 erstellt"
Warning message:
"Paket 'lubridate' wurde unter R Version 4.2.3 erstellt"
── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────

In [2]:
# Daten laden
tria_beh_mon <- c(
	c(58000, 276100, 111700, 20400),
	c(0, 100900, 480319, 194319),
	c(0, 0, 120800, 575050),
	c(0, 0, 0, 130900)# * 2
) |>
	matrix(nrow = 4) |>
	t()

tria_beh_mon

58000,276100,111700,20400
0,100900,480319,194319
0,0,120800,575050
0,0,0,130900


In [3]:
# In timelag-Ansicht wechseln
tria_rel <- tria_beh_mon
for (i in 1:nrow(tria_rel)) tria_rel[i,] <- lead(tria_rel[i,], i - 1)
tria_rel

58000,276100,111700,20400
100900,480319,194319,NA
120800,575050,NA,NA
130900,NA,NA,NA


In [4]:
# Kumulierte Ansicht
tria_kum <- tria_rel
for (j in 2:ncol(tria_kum)) tria_kum[,j] <- tria_kum[,j] + tria_kum[,j - 1]
tria_kum

58000,334100,445800,466200
100900,581219,775538,NA
120800,695850,NA,NA
130900,NA,NA,NA


In [5]:
# Faktoren berechnen
tria_kum_prev <- tria_kum
tria_kum_prev[!is.na(tria_kum_prev)] <- 1
for (i in 1:nrow(tria_kum_prev)) tria_kum_prev[i,] <- lead(tria_kum_prev[i,], 1)
tria_kum_prev <- tria_kum_prev * tria_kum

factors <- c()
for (j in 2:ncol(tria_kum)) {
	sum_prev <- sum(tria_kum_prev[,j - 1], na.rm = TRUE)
	sum_base <- sum(tria_kum[,j], na.rm = TRUE)
	factors <- append(factors, sum_base / sum_prev)
}
factors

[1] 5.760347 1.334330 1.045760

In [6]:
# Abrechnungsquadrat rechnen
quadr_kum <- tria_kum
for (j in 2:ncol(quadr_kum)) quadr_kum[,j] <- if_else(is.na(quadr_kum[,j]), factors[j - 1] * quadr_kum[,j - 1], quadr_kum[,j])
quadr_kum

58000,334100.0,445800.0,466200.0
100900,581219.0,775538.0,811027.0
120800,695850.0,928493.8,970982.1
130900,754029.4,1006124.4,1052165.1


In [7]:
# nicht kumulierte Darstellung
quadr <- quadr_kum
for (j in 2:ncol(quadr)) quadr[,j] <- quadr_kum[,j] - quadr_kum[,j - 1]
quadr

58000,276100.0,111700.0,20400.00
100900,480319.0,194319.0,35488.95
120800,575050.0,232643.8,42488.28
130900,623129.4,252095.0,46040.68


In [8]:
# Ansicht nach abrechnungsmonaten
abr_mon <- cbind(quadr, matrix(NA_real_,4,3))
for (i in 1:nrow(abr_mon)) abr_mon[i,] <- lag(abr_mon[i,], i - 1)
abr_mon

58000,276100,111700,20400,NA,NA,NA
NA,100900,480319,194319,35488.95,NA,NA
NA,NA,120800,575050,232643.83,42488.28,NA
NA,NA,NA,130900,623129.40,252094.98,46040.68


In [9]:
# Summen über Abrechnugnsmonate
abr_mon_tbl <- abr_mon |>
	as_tibble() |>
	mutate(behandlungsmonat = 1:n()) |>
	pivot_longer(
		starts_with("V"),
		names_to = "abrechnungsmonat",
		values_to = "betrag",
		names_prefix = "V"
	) |>
	mutate(across(behandlungsmonat | abrechnungsmonat, as.integer)) |>
	filter(!is.na(betrag))

abr_mon_tbl
abr_mon_tbl |>
	group_by(abrechnungsmonat) |> 
	summarise(across(betrag, sum), .groups = "drop")
abr_mon_tbl |>
	filter(abrechnungsmonat > 4) |> 
	summarise(across(betrag, sum), .groups = "drop")

Warning message:
"The `x` argument of `as_tibble.matrix()` must have unique column names if
`.name_repair` is omitted as of tibble 2.0.0.
ℹ Using compatibility `.name_repair`."


behandlungsmonat,abrechnungsmonat,betrag
<int>,<int>,<dbl>
1,1,58000.00
1,2,276100.00
1,3,111700.00
1,4,20400.00
2,2,100900.00
2,3,480319.00
2,4,194319.00
2,5,35488.95
3,3,120800.00


abrechnungsmonat,betrag
<int>,<dbl>
1,58000.00
2,377000.00
3,712819.00
4,920669.00
5,891262.18
6,294583.25
7,46040.68


betrag
<dbl>
1231886


> Somit müssen 1'231'886 Franken Rückstellungen gebildet werden.

b) Angenommen in den darauffolgenden Behandlungsmonaten 5 und 6 fallen Leistungen von total 1'000'000 (Behandlungsmonat 5) bzw. 1'100'000 (Behandlungsmonat 6) an. Das Abwicklungsmuster bleibe dasselbe wie in den vorangehenden Monaten 1 bis 4. Wie viele Leistungen werden in den Abrechnungsmonaten 5 und 6 abgerechnet?

> Die Faktoren `factors`, $\phi_j$ bilden das kumulierte relative Abrechnungsvolumen gegenüber dem Vormonat ab. Gehen wir wieder davon aus, dass nach vier Monaten alle Leistungen abgerechnet sind, dann kann man die kumulierten Anteile der Abrechnung (*Quoten*) als Kehrwert der Faktoren bestimmen.

$$\gamma_j = \gamma_{j + 1} / \phi_{j + 1} \text{ mit } \gamma_n = 1$$

In [10]:
# Abwicklungsquoten
quotes <- c(1)
for (j in (length(factors)):1) quotes <- append(quotes[1] / factors[j], quotes)
quotes

[1] 0.1244101 0.7166455 0.9562420 1.0000000

In [11]:
# Abwicklungsanteile
anteile <- quotes
for (j in 2:length(quotes)) anteile[j] <- quotes[j] - quotes[j - 1]
anteile

[1] 0.12441014 0.59223540 0.23959642 0.04375804

> Damit werden von den 1'000'000 behandelten Leistungen im Monat 5 124'410 im Monat 5 und 592'235 im Monat 6 abgerechnet. Von den 1'100'000 behandelten Leistungen im Monat 6 werden 136'851 im Monat 6 abgerechnet.

In [12]:
abr_mon_tbl_new <- abr_mon_tbl |>
	union_all(
		tibble(
			behandlungsmonat = 5,
			abrechnungsmonat = 5:8,
			betrag = anteile * 1000000
		) 
	) |>
	union_all(
		tibble(
			behandlungsmonat = 6,
			abrechnungsmonat = 6:9,
			betrag = anteile * 1100000
		) 
	)
abr_mon_tbl_new
abr_mon_tbl_new |> group_by(abrechnungsmonat) |> summarise(across(betrag, sum))

behandlungsmonat,abrechnungsmonat,betrag
<dbl>,<int>,<dbl>
1,1,58000.00
1,2,276100.00
1,3,111700.00
1,4,20400.00
2,2,100900.00
2,3,480319.00
2,4,194319.00
2,5,35488.95
3,3,120800.00


abrechnungsmonat,betrag
<int>,<dbl>
1,58000.00
2,377000.00
3,712819.00
4,920669.00
5,1015672.32
6,1023669.81
7,937096.04
8,307314.10
9,48133.85


c) Schildern Sie in ein paar wenigen Stichworten, wie sich Chain-Ladder nicht bloss als Algorithmus sondern auch als stochastisches Modell implementieren lässt. 